In [ ]:
import numpy as np
from pandas import *
import random

class MyLineReg():
  def __init__(self,weigths=None,n_iter=100,learning_rate=None,metric=None,y_pred=None,reg=None,l1_coef=0,l2_coef=0,sgd_sample=None,random_state=42):
    self.n_iter=n_iter
    self.learning_rate=learning_rate
    self.weights=weigths
    self.metric=metric
    self.y_pred=y_pred
    self.reg=reg
    self.l1_coef=l1_coef
    self.l2_coef=l2_coef
    self.sgd_sample=sgd_sample
    self.random_state=random_state
  def __str__(self):
    return "MyLineReg class: "+ ", ".join(('{}={}'.format(item, self.__dict__[item]) for item in self.__dict__))
  def __repr__(self):
     return "MyLineReg class: "+ ", ".join(('{}={}'.format(item, self.__dict__[item]) for item in self.__dict__))
  def fit(self,X,y,verbose=False):

    random.seed(self.random_state)
    self.weights=np.ones(X.shape[1]+1)
    X.insert(0,"ones",np.ones(X.shape[0]))
    metrica=None
    y_pred=None
    is_double=isinstance(self.learning_rate, float)
    X.reset_index(drop=True,inplace=True)
    y.reset_index(drop=True,inplace=True)
    for i in range(1,self.n_iter+1):

      if isinstance(self.sgd_sample,int):
        sample_rows_idx = random.sample(range(X.shape[0]), self.sgd_sample)
      elif self.sgd_sample != None:
        sample_rows_idx = random.sample(range(X.shape[0]), int(self.sgd_sample*len(y)))
      else:
        sample_rows_idx=X.index

      X_cut = X.iloc[sample_rows_idx]
      y_cut = y.iloc[sample_rows_idx]

      y_pred = self.weights @ X_cut.T

      if self.reg == "l1":
        MSE = np.mean((y-y_pred)**2) + self.l1_coef * np.sum(self.weights)
        gradient = -2 *  X_cut.T @ (y_cut-y_pred)/len(y_cut) + self.l1_coef * np.sign(self.weights)
      elif self.reg=="l2":
        MSE = np.mean((y-y_pred)**2) + self.l2_coef * np.sum(self.weights**2)
        gradient = -2 *  X_cut.T  @ (y_cut-y_pred)/len(y_cut) + self.l2_coef * 2 * self.weights
      elif self.reg=="elasticnet":
        MSE = np.mean((y-y_pred)**2) + self.l1_coef * np.sum(self.weights) + self.l2_coef * np.sum(self.weights**2)
        gradient = -2 *  X_cut.T  @ (y_cut-y_pred)/len(y_cut) + self.l1_coef * np.sign(self.weights) + self.l2_coef * 2 * self.weights
      else:
        MSE = np.mean((y-y_pred)**2)
        gradient = -2 *  X_cut.T  @ (y_cut-y_pred)/len(y_cut)

      if not is_double:
        self.weights-=self.learning_rate(i) * gradient
      else:
        self.weights-=self.learning_rate * gradient

      if verbose and (i+1)%verbose==0:
        if self.metric!=None:
          print(f"{i+1} | loss : {MSE: .2f} | {self.metric}:{metrica}")
        else:
           print(f"{i+1} | loss : {MSE: .2f}")

    self.y_pred=self.weights @ X.T


  def get_coef(self):
    return self.weights[1:]

  def predict(self,X):
    X.insert(0,"ones",np.ones(X.shape[0]))
    return np.mean(X @ self.weights)

  def get_best_score(self):
    y_pred = self.y_pred
    if self.metric=="mae":
      return np.mean(abs(y-y_pred))
    elif self.metric == 'mse':
      return np.mean((y-y_pred)**2)
    elif self.metric=="rmse":
       return (np.mean((y-y_pred)**2))**(1/2)
    elif self.metric =="mape":
      return 100*np.mean(abs((y-y_pred)/y))
    elif self.metric=="r2":
      y_sred=np.mean(y)
      return 1- (np.sum((y-y_pred)**2))/(np.sum((y-y_sred)**2))
a=MyLineReg(learning_rate=2.0,sgd_sample=0.1)
y=Series([1])
a.fit(DataFrame(np.random.randn(1, 1)),y)
a.predict(DataFrame(np.random.randn(1,1)))


nan